# Registration example using multiscale-imaging
This notebook contains an example of applying the registration with the functions transferred into the multiscale-imaging package.

In [1]:
from pathlib import Path
import numpy as np
import SimpleITK as sitk
%matplotlib notebook
import matplotlib.pyplot as plt
import math
import pandas as pd
import tiffile as tif
import pickle
import os

import multiscale.microscopy.ome as ome
import multiscale.LINK_system.coordinate as coord
import multiscale.itk.transform as tran
import multiscale.itk.itk_plotting as iplt

# Set up registration
First define the paths to each needed file.  This includes the transform parameters, the images themselves, as well as the US position list and the first tile of the microscopy image.

In [12]:
# The input files are available at https://uwmadison.box.com/s/05ou19dsi2y2wfzgocf1ab7cbbeb4fgn
fiducial_dir = Path(r'F:\Research\LINK\System calibration\2019-10-04 - 4x and 20x fiducial')
sample_dir = Path(r'F:\Research\LINK\Eye images\2019-10-08 - Agarose eye\Eye images')

registered_dir = Path(sample_dir, 'Registered')

In [13]:
microscopy_path = Path(sample_dir, 'SHG down4x.tif')
microscopy_tile_path = Path(sample_dir, 'SHG settings.ome.tif')

oct_path = Path(sample_dir, 'OCT.tif')
oct_pl_path = Path(sample_dir, 'OCT Position.pos')

transform_path = Path(fiducial_dir, 'OCT to SHG.txt')

Now open the image.  If the spacing information is not properly written to either image it can still be set manually.

In [4]:
spacing = [2, 2, 7.8]
zero_reference = -2600
us_image = coord.open_oct(oct_path, oct_pl_path, spacing, zero_reference)

In [5]:
microscopy_image = coord.open_microscopy(microscopy_path, microscopy_tile_path, downsample_factor=4)

Load the coordinate transform

In [6]:
with open(transform_path, 'rb') as fp:
    coordinate_transform = pickle.load(fp)

Apply the registration and save the results

In [7]:
coordinate_transform[5] = coordinate_transform[5]

In [10]:
oct_reg = tran.apply_transform_params(microscopy_image, oct_image, 
                                     coordinate_transform,
                                     sitk.VersorRigid3DTransform())

In [ ]:
oct_reg_path = Path(registered_dir, 'OCT registered to SHG.tif')

In [16]:
sitk.WriteImage(oct_reg, str(oct_reg_path))